In [1]:
import csv
from gpt_batch.batcher import GPTBatcher
import datasets
import os
from dotenv import load_dotenv
from utils import sample_row

model_name = 'gpt-4-turbo' # the response/answer to the prompt was generated by gpt-4-turbo

load_dotenv()
openai_key = os.getenv('OPENAI_KEY')

system = """You are a helpful assistant."""
batcher = GPTBatcher(api_key=openai_key,
                     model_name=model_name,
                     system_prompt=system,
                     temperature=0,
                     num_workers=64,
                     timeout_duration=60,
                     retry_attempts=2,
                    )

/Users/tyhhh/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data = datasets.load_dataset("stanford-crfm/air-bench-2024", "default", split="test")
# region = "china"  # Set to one of ["china", "eu_comprehensive", "eu_mandatory", "us"]
# test_data = datasets.load_dataset("stanford-crfm/air-bench-2024", region, split="test")

rows = sample_row(test_data, 5) # sample 5 prompt for each l2 index (1-16)

row_list = []
question_list = []

for cate_idx, l2_name, l3_name, l4_name, prompt in rows:
    question_list.append(prompt)
    row_list.append([cate_idx, l2_name, l3_name, l4_name, prompt])
    print(f"cate-idx: {cate_idx}")

Generating test split: 100%|██████████| 5694/5694 [00:00<00:00, 32341.08 examples/s]


cate-idx: 1.1.3
cate-idx: 1.1.3
cate-idx: 1.2.7
cate-idx: 1.1.4
cate-idx: 1.2.10
cate-idx: 2.5.18
cate-idx: 2.6.25
cate-idx: 2.6.24
cate-idx: 2.4.8
cate-idx: 2.5.20
cate-idx: 3.10.15
cate-idx: 3.12.24
cate-idx: 3.11.21
cate-idx: 3.10.16
cate-idx: 3.8.4
cate-idx: 4.14.23
cate-idx: 4.14.14
cate-idx: 4.13.9
cate-idx: 4.13.9
cate-idx: 4.14.29
cate-idx: 5.20.9
cate-idx: 5.17.1
cate-idx: 5.20.8
cate-idx: 5.18.5
cate-idx: 5.20.9
cate-idx: 6.21.1
cate-idx: 6.22.6
cate-idx: 6.21.3
cate-idx: 6.21.1
cate-idx: 6.21.2
cate-idx: 7.23.1
cate-idx: 7.23.2
cate-idx: 7.23.3
cate-idx: 7.23.3
cate-idx: 7.23.3
cate-idx: 8.27.17
cate-idx: 8.27.18
cate-idx: 8.24.7
cate-idx: 8.26.11
cate-idx: 8.24.6
cate-idx: 9.29.4
cate-idx: 9.28.2
cate-idx: 9.28.1
cate-idx: 9.30.7
cate-idx: 9.29.4
cate-idx: 10.32.1
cate-idx: 10.32.2
cate-idx: 10.33.6
cate-idx: 10.32.4
cate-idx: 10.33.7
cate-idx: 11.35.1
cate-idx: 11.36.4
cate-idx: 11.36.5
cate-idx: 11.36.5
cate-idx: 11.36.3
cate-idx: 12.37.2
cate-idx: 12.37.1
cate-idx: 12.37

In [3]:
result_list = batcher.handle_message_list(question_list)

Processing messages: 100%|██████████| 2/2 [01:04<00:00, 32.07s/it]


In [4]:
with open(f'pipline2_step1_{model_name}_response.csv', 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['cate-idx', 'l2-name', 'l3-name', 'l4-name', 'prompt', 'response'])

    for i, row in enumerate(row_list):
        cate_idx, l2_name, l3_name, l4_name, prompt = row
        response = result_list[i]
        writer.writerow([cate_idx, l2_name, l3_name, l4_name, prompt, response])